# NetworKit Graph Tutorial

In this notebook we will cover the main functionalities of `networkit.Graph`, the central class in NetworKit. The first step is to import NetworKit.

In [ ]:
import networkit as nk

We start by creating the core object, a `networkit.Graph`. The [networkit.Graph](https://networkit.github.io/dev-docs/python_api/networkit.html?highlight=graph#networkit.Graph) constructor expects the `number of nodes` as an integer, a boolean value stating if the graph is weighted or not followed by another boolean value stating whether the graph is directed or not. The latter two are set to false by default. If the graph is unweighted, all edge weights are set to `1.0`.

In [ ]:
G = nk.Graph(5)
print(G.numberOfNodes(), G.numberOfEdges())
print(G.isWeighted(), G.isDirected())

`G` has 5 nodes, but no edges yet. Using the [addEdge(node u, node v)](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=addedge#networkit.graph.Graph.addEdge) method, we can add edges between the nodes.

In [ ]:
G.addEdge(1, 3)
G.addEdge(2, 4)
G.addEdge(1, 2)
G.addEdge(3, 4)
G.addEdge(2, 3)
G.addEdge(4, 0)

Node IDs in NetworKit are integer indices that start at 0 through to `G.upperNodeIdBound() - 1`. Hence, for this graph `G.addEdge(1,5)` would an illegal operation as node `5` does not exist. The same goes for edge IDs. If we need to add an edge between node 0 and node 5, we first need to add a sixth node to the graph using `G.addNode()`.

In [ ]:
G.addNode()
print(G.numberOfNodes())

Now we can add the new edge.

In [ ]:
G.addEdge(0,5)

Using the method [overview(G)](https://networkit.github.io/dev-docs/python_api/networkit.html?highlight=overview#networkit.overview), we can take a look at the main properties of the graph we have created.

In [ ]:
nk.overview(G)

Now that we have created a graph we can start to play around with it. Say we want to remove the node with the node ID 2, so the third node. We can easily do so using [Graph.removeNode(node u)](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=remove%20node#networkit.graph.Graph.removeNode). 

In [ ]:
G.removeNode(2)

In [ ]:
# 2 has been deleted
print(G.hasNode(2))

The node has been remove from the graph, however, the node IDs are not adjusted to the match the new number of nodes. Hence, if we want to restore the node we previously removed from G, we can do so using [Graph.restoreNode(node u)](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=restore#networkit.graph.Graph.restoreNode) using the original node ID.

In [ ]:
# Restore node with ID 2
G.restoreNode(2)

# Check if it is back in G
print(G.hasNode(2))

Note that in default mode NetworKit allows you to add an edge multiple times, most algorithms (and also io-functions) do not support it.

In [ ]:
G.addNode()
G.addEdge(0, 6)
print(G.numberOfEdges())
# NetworKit does not complain when inserting the same edge a second time 
G.addEdge(0, 6)
print(G.numberOfEdges())

If wanted, this behavior can be changed. This increases the running time of adding an edge by the degree of the involved nodes.

In [ ]:
# Remove one of the multiple edges
G.removeEdge(0, 6)
print(G.numberOfEdges())
# The multi-edge is not added to the graph. 
G.addEdge(0, 6, checkMultiEdge = True)
print(G.numberOfEdges())

NetworKit provides iterators that enable iterating over all nodes or edges in a simple manner. There are two kinds of iterators: one is based on ranges, the other one accepts callback a function.
The easiest to use are the range-based iterators, they can be used in a simple for loop:

In [ ]:
# Iterate over the nodes of G
for u in G.iterNodes():
    print(u)
    if u > 4:
        print('...')

In [ ]:
# Iterate over the edges of G
for u, v in G.iterEdges():
    print(u, v)
    if u > 2:
        print('...')

In [ ]:
# Iterate over the edges of G1 and include weights
G1 = nk.graphtools.toWeighted(G)
G1.setWeight(0, 4, 2)
G1.setWeight(1, 3, 3)
for u, v, w in G1.iterEdgesWeights():
    print(u, v, w)
    if u > 2:
        print('...')

Callback-based iterators accept a callback function passed to the iterators as input parameter.
Those functions can also be lambda functions.
More information can be found in the NetworKit documentation [here](https://networkit.github.io/dev-docs/python_api/graph.html). Let's start by using the [forNodes](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=fornodes#networkit.graph.Graph.forNodes) iterator. It expects a callback function which accepts one parameter, i.e., a node. First, we define such a function.

In [ ]:
def nodeFunc(u):
    print("Node ", u, " passed to nodeFunc()")

We then pass `nodeFunc` to the iterator. 

In [ ]:
G.forNodes(nodeFunc)

In [ ]:
G.forNodes(lambda u: print("Node ", u, " passed to lambda"))

Similarly, we can iterate over the edges of `G`:

In [ ]:
# First define callback function
# that accepts exactly 4 parameters:
def edgeFunc(u, v, weight, edgeId):
    print("Edge from {} to {} has weight {} and id {}".format(u, v, weight, edgeId))

We can now call the [forEdges](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=foredges#networkit.graph.Graph.forEdges) iterator, and pass `edgeFunc` to it.

In [ ]:
# Using iterator with callback function.
G.forEdges(edgeFunc)

Although we did not add any indexes to our edges, our edges all have indexes of 0. This is because NetworKit by default indexes all edges with 0. However, sometimes it makes sense to have indexed edges. If you decide to index the edges of your graph after creating it, you can use the [Graph.indexEdges(bool force = False)](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=indexedges#networkit.graph.Graph.indexEdges) method. The `force` parameter forces re-indexing of edges if they had already been indexed.

Since we did not index the edges of our graph initially, we can use the default value. Indexing the edges of `G` can then be done as simply as follows:

In [ ]:
G.indexEdges()

Sometimes you also need to iterate over specific edges, for example the ones connecting a node `u` to its neighbors. Using the [forNodes](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=foredges#networkit.graph.Graph.forNodes) iterator and the [forEdgesOf](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=foredges#networkit.graph.Graph.forEdges) iterator we can do so.

In [ ]:
G.forNodes(lambda u: G.forEdgesOf(u, edgeFunc))

Note, in an undirected graph, like we have here, the [forEdgesOf](https://networkit.github.io/dev-docs/python_api/networkit.html?highlight=foredgesof#networkit.Graph.forEdgesOf) iterator returns all edges of a node. When dealing with a directed graph only the out edges are returned. The rest of the edges can be accessed using the [forInEdgesOf](https://networkit.github.io/dev-docs/python_api/networkit.html?highlight=forinedges#networkit.Graph.forInEdgesOf) iterator.

## Node Attributes

It is possible to attach attributes to the nodes of a NetworKit graph with `attachNodeAttribute`. Attributes can be of type `str`, `float`, or `int`.

In [ ]:
# Create a new attribute named 'myAtt' of type 'str'
att = G.attachNodeAttribute("myAtt", str)

# Set attribute values
att[0] = "foo" # Attribute of node 0
att[1] = "bar" # Attribute of node 1

# Get attribute value
for u in G.iterNodes():
    try:
        print(f"Attribute of node {u} is {att[u]}")
    except ValueError:
        print(f"Node {u} has no attribute")
        break    

## GraphTools

`networkit.graphtools` implements some useful functions to get information or modify graphs. The following section shows some of the GraphTools functionalities.

`toWeighted(G)` takes an unweighted graph `G` as input, and returns a weighted copy of `G`. All the edge weights are set to a default value of 1.0.

In [ ]:
weightedG = nk.graphtools.toWeighted(G)
assert(weightedG.numberOfNodes() == G.numberOfNodes())
assert(weightedG.numberOfEdges() == G.numberOfEdges())
assert(weightedG.isWeighted())

`toUnweighted(G)` does the inverse of the one above: it takes a weighted graph `G` as input, and returns an unweighted copy of `G` as output.

`randomNode(G)` returns a node of `G` selected uniformly at random.

In [ ]:
nk.graphtools.randomNode(G)

`randomNeighbor(G, u)` returns a random (out) neighbor of node `u` in the graph `G`.

In [ ]:
nk.graphtools.randomNeighbor(G, 0)

`randomEdge(G, uniformDistribution=False)` returns a random edge of graph `G`. If `uniformDistribution` is set to `True`, the edge is selected uniformly at random.

In [ ]:
nk.graphtools.randomEdge(G, True)

Sometimes it makes sense to compact the graph, e.g., after deleting nodes. The method `getCompactedGraph(G, nodeIdMap)` does just that by designating continuous node ids. `nodeIdMap` maps each node id of graph `G` to their new ids.

First, we delete a node from `G`.

In [ ]:
G.removeNode(2)

Then, we use `getContinuousNodeIds(G)` to get a map from the original nodes ids of G, to their new ids.

In [ ]:
nodeIdMap = nk.graphtools.getContinuousNodeIds(G)

Finally, we get a new graph with compacted node ids.

In [ ]:
compGraph = nk.graphtools.getCompactedGraph(G, nodeIdMap)
assert(compGraph.numberOfNodes() == G.numberOfNodes())
assert(compGraph.numberOfEdges() == G.numberOfEdges())

`maxDegree(G)` returns the highest (out) degree of `G`.

In [ ]:
nk.graphtools.maxDegree(G)

`maxInDegree(G)` returns the highest in-degree of a directed graph `G`. `maxWeightedDegree(G)` returns the highest sum of the (out) edge weights of `G`, while `maxWeightedInDegree(G)` returns the highest sum of the in-edge weights of a directed graph `G`.